In [1]:
import numpy as np

import sys
from pathlib import Path

import torch

from sklearn.model_selection import train_test_split
import wandb

In [2]:
# Importerer spaleron og micro svarte hull data

module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
import trainers
import nnmodels

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

Running on the GPU


In [3]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [4]:
print(bhArray.shape)
print(sphArray.shape)

(15000, 50, 50, 3)
(15000, 50, 50, 3)


In [5]:
# Vi slår sammen dataene for å ha et samlet datasett som kjører gjennom modellen
dataArray = np.concatenate((bhArray,sphArray),axis=0)

In [6]:
# Labeler tabelle med 1 og 0 (0 = svart hull, 1 = spahleron)
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [8]:
# 75% av datasettet havner i trainData, 25% havner i testData, 
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [9]:
# gjør det om til en tensor fra numpy array. Vi gjør dette for at at dataene skal være lagret på GPU en istedet for en liste som er lagret på CPUen 
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [10]:
# to tensor datasett 
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [11]:
from torchvision import models
from torchsummary import summary

model = nnmodels.ConvModel(0).to(device)
summary(model, (3, 50, 50))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 48, 48]             448
            Conv2d-2           [-1, 64, 22, 22]           9,280
            Conv2d-3            [-1, 256, 9, 9]         147,712
            Linear-4                 [-1, 1000]       2,305,000
           Dropout-5                 [-1, 1000]               0
            Linear-6                    [-1, 2]           2,002
Total params: 2,464,442
Trainable params: 2,464,442
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.69
Params size (MB): 9.40
Estimated Total Size (MB): 10.12
----------------------------------------------------------------


In [12]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'Test accuracy'
        },
    'parameters': {
        'model': {'values': ['resnet34', 'resnet34Pre', 'resnet50', 'resnet50Pre']},
        'loss': {'values': ['cross', 'hinge', 'customLoss-3', 'customLoss-4', 'customLoss-5', 'customLoss-6', 'customLoss-7']},
        'epoch': {'values': [10]},
        'batch_size': {'values': [50]},
        'gamma': {'values': [1]},
        'learning_rate': {'values': [0.01]},
        'optimizer': {'values': ['adam', 'sgd']},
        'dropout': {'values': [0]},
     }
}

In [13]:
#wrapper function
def trainFunction():
    with wandb.init(project="Sweep", name="Sweep") as run:
        config = wandb.config

        name_str = config['model'] + "_" + config['loss'] + "_" + config['optimizer']
        run.name = name_str
        
        trainers.sweep(train, test, config)

In [14]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project="Sweep")
wandb.agent(sweep_id, function=trainFunction)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: rfg5zt1x
Sweep URL: https://wandb.ai/g13hvl2023/Sweep/sweeps/rfg5zt1x


wandb: Agent Starting Run: nty1mhce with config:
wandb: 	batch_size: 50
wandb: 	dropout: 0
wandb: 	epoch: 10
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: cross
wandb: 	model: resnet34
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 591291 (g13hvl2023). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


BH accuracy,▁▇▇▇▇█▇██▇▇
SPH accuracy,█▇█▇█▇▇▁▇██
Test accuracy,▁▇▇████▃███
Test epoch_loss,▂▁▁▁▁▁▁█▁▁▁
Test precision,█▇█▇█▇▇▁▇██
Test recall,▁▆▆▇▇█▇█▇▇▆
Train accuracy,▁▅▇▇███▇██
Train epoch_loss,█▂▂▁▁▁▁▂▁▁
BH accuracy,0.80058
SPH accuracy,0.97573
Test accuracy,0.8872


wandb: Agent Starting Run: ehqn6kwc with config:
wandb: 	batch_size: 50
wandb: 	dropout: 0
wandb: 	epoch: 10
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: cross
wandb: 	model: resnet34
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: Agent Starting Run: xe72cita with config:
wandb: 	batch_size: 50
wandb: 	dropout: 0
wandb: 	epoch: 10
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: hinge
wandb: 	model: resnet50Pre
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


c:\Users\student\Anaconda3\envs\DAT191-gpu\lib\site-packages\torch\autograd\__init__.py:197: UserWarning: An output with one or more elements was resized since it had shape [], which does not match the required output shape [50, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Resize.cpp:24.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
c:\Users\student\Anaconda3\envs\DAT191-gpu\lib\site-packages\torch\autograd\__init__.py:197: UserWarning: An output with one or more elements was resized since it had shape [], which does not match the required output shape [50, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements

BH accuracy,█▅▁▅▇▆▆▆▇▆▇
SPH accuracy,▁▇▃█▇█████▇
Test accuracy,▃▇▁▇███████
Test epoch_loss,█▁▃▁▁▁▁▁▁▁▁
Test precision,▁▇▃█▇█████▇
Test recall,▄▆▁▇█▇▇▇█▇█
Train accuracy,▁▅▆▇▇█████
Train epoch_loss,█▁▂▁▁▁▁▁▁▁
BH accuracy,0.91664
SPH accuracy,0.82205
Test accuracy,0.86987


wandb: Agent Starting Run: 1tvpao9h with config:
wandb: 	batch_size: 50
wandb: 	dropout: 0
wandb: 	epoch: 10
wandb: 	gamma: 1
wandb: 	learning_rate: 0.01
wandb: 	loss: hinge
wandb: 	model: resnet50Pre
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


c:\Users\student\Anaconda3\envs\DAT191-gpu\lib\site-packages\torch\autograd\__init__.py:197: UserWarning: An output with one or more elements was resized since it had shape [], which does not match the required output shape [50, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Resize.cpp:24.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
c:\Users\student\Anaconda3\envs\DAT191-gpu\lib\site-packages\torch\autograd\__init__.py:197: UserWarning: An output with one or more elements was resized since it had shape [], which does not match the required output shape [50, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements